In [ ]:
import gymnasium as gym
import pandas as pd
import numpy as np
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from sklearn.preprocessing import StandardScaler

# ✅ 1️⃣ Load stock data and preprocess
df = pd.read_csv("/home/jesse/Projects/RL_Testing/SAC_Testing/TandT/nvidia_stock_with_indicators.csv",
                 index_col="Date", parse_dates=True)

# Normalize the data to prevent gradient explosion
scaler = StandardScaler()
df[df.columns] = scaler.fit_transform(df[df.columns])  

# ✅ 2️⃣ Define the stock trading environment
class StockTradingEnv(gym.Env):
    def __init__(self, df, window_size=30, initial_balance=10000, transaction_cost=0.001):
        super(StockTradingEnv, self).__init__()
        self.df = df
        self.window_size = window_size
        self.initial_balance = initial_balance
        self.transaction_cost = transaction_cost  # Transaction cost factor
        self.current_step = self.window_size

        # Account variables
        self.balance = initial_balance
        self.shares_held = 0
        self.total_asset_value = initial_balance

        # Observation space
        self.feature_columns = [col for col in df.columns if col != "Date"]
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, len(self.feature_columns) + 1), dtype=np.float32
        )

        # Action space: -1 (sell) to 1 (buy)
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)

    def _next_observation(self):   
        current_price = self.df.iloc[self.current_step]["Close"]  # Define current price first
        features = self.df.iloc[self.current_step - self.window_size:self.current_step][self.feature_columns].values
        
        # Ensure current price is nonzero to prevent division errors
        position_ratio = np.array([[self.shares_held / (self.total_asset_value / (current_price + 1e-8))]] * self.window_size)

        return np.hstack([features, position_ratio]).astype(np.float32)

    def step(self, action):
        prev_asset_value = self.total_asset_value
        current_price = self.df.iloc[self.current_step]["Close"]

        # Compute trade amount considering available balance
        trade_amount = action[0] * self.balance  
        
        if trade_amount > 0:  # Buy
            trade_amount = min(trade_amount, self.balance)  # Prevent overspending
            shares_bought = trade_amount / current_price
            self.shares_held += shares_bought
            self.balance -= trade_amount
        
        elif trade_amount < 0:  # Sell
            shares_sold = min(abs(trade_amount) / current_price, self.shares_held)
            self.shares_held -= shares_sold
            self.balance += shares_sold * current_price

        # Update total asset value
        self.total_asset_value = self.balance + (self.shares_held * current_price)
        
        # Compute reward (return - transaction cost)
        reward = (self.total_asset_value - prev_asset_value) / (prev_asset_value + 1e-8)  # Avoid Zero-Division
        cost = self.transaction_cost * abs(action[0]) / self.total_asset_value  
        reward -= cost  

        self.current_step += 1
        terminated = self.current_step >= len(self.df) - 1
        truncated = False  

        return self._next_observation(), reward, terminated, truncated, {}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = self.window_size
        self.balance = self.initial_balance
        self.shares_held = 0
        self.total_asset_value = self.initial_balance
        return self._next_observation(), {}  

    def render(self):
        print(f"Step: {self.current_step}, Balance: {self.balance:.2f}, Portfolio Value: {self.total_asset_value:.2f}")

# ✅ 3️⃣ Create a vectorized environment for training
def create_env():
    return StockTradingEnv(df)  

env = make_vec_env(create_env, n_envs=1)

print("✅ Trading environment successfully created!")


✅ Trading environment successfully created!


In [7]:
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
import pandas as pd

# ✅ 4️⃣ Train SAC agent
policy_kwargs = dict(net_arch=[512, 512, 256])  # Deeper network for better feature extraction
model = SAC(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=3e-5,  # Optimized learning rate
    ent_coef=0.01,  # Controls exploration-exploitation tradeoff
    policy_kwargs=policy_kwargs,
    batch_size=512,  # Larger batch size for efficient GPU utilization
    tensorboard_log="./sac_logs/"
)

# ✅ 5️⃣ Train for 200,000 steps
model.learn(total_timesteps=200000)

# ✅ 6️⃣ Save trained model
model.save("sac_NVDA")

# ✅ 7️⃣ Evaluate after training
print("🎉 Training complete! Model saved as 'sac_NVDA' 🎉")

Using cuda device
Logging to ./sac_logs/SAC_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4.5e+03  |
|    ep_rew_mean     | -19.1    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 48       |
|    time_elapsed    | 369      |
|    total_timesteps | 18012    |
| train/             |          |
|    actor_loss      | 1.74     |
|    critic_loss     | 0.0137   |
|    ent_coef        | 0.01     |
|    learning_rate   | 3e-05    |
|    n_updates       | 17911    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4.5e+03  |
|    ep_rew_mean     | -0.215   |
| time/              |          |
|    episodes        | 8        |
|    fps             | 44       |
|    time_elapsed    | 801      |
|    total_timesteps | 36024    |
| train/             |          |
|    actor_loss      | 0.854    |
|    critic_loss     | 0.045    |
| 

In [8]:
import pandas as pd

# Load stock data
msft_df = pd.read_csv("/home/jesse/Projects/RL_Testing/SAC_Testing/TandT/microsoft_stock_with_indicators.csv")

# Initialize the trading environment
env = StockTradingEnv(msft_df)
obs, _ = env.reset()  # `gymnasium` requires reset() to return (obs, info)

# Run the trained model in the environment
for _ in range(len(msft_df)):
    action, _states = model.predict(obs)
    obs, reward, terminated, truncated, _ = env.step(action)  # `gymnasium` returns 5 values
    env.render()
    
    if terminated or truncated:
        break


Step: 31, Balance: 10000.00, Portfolio Value: 10000.00
Step: 32, Balance: 10000.00, Portfolio Value: 10000.00
Step: 33, Balance: 10000.00, Portfolio Value: 10000.00
Step: 34, Balance: 10000.00, Portfolio Value: 10000.00
Step: 35, Balance: 10000.00, Portfolio Value: 10000.00
Step: 36, Balance: 10000.00, Portfolio Value: 10000.00
Step: 37, Balance: 10000.00, Portfolio Value: 10000.00
Step: 38, Balance: 10000.00, Portfolio Value: 10000.00
Step: 39, Balance: 10000.00, Portfolio Value: 10000.00
Step: 40, Balance: 10000.00, Portfolio Value: 10000.00
Step: 41, Balance: 10000.00, Portfolio Value: 10000.00
Step: 42, Balance: 0.00, Portfolio Value: 10000.00
Step: 43, Balance: 0.00, Portfolio Value: 10272.74
Step: 44, Balance: 0.00, Portfolio Value: 10386.37
Step: 45, Balance: 0.00, Portfolio Value: 10363.64
Step: 46, Balance: 0.00, Portfolio Value: 10409.09
Step: 47, Balance: 0.00, Portfolio Value: 10409.09
Step: 48, Balance: 0.00, Portfolio Value: 10363.64
Step: 49, Balance: 0.00, Portfolio Val

In [9]:
import pandas as pd

# Load stock data
aapl_df = pd.read_csv("/home/jesse/Projects/RL_Testing/SAC_Testing/TandT/apple_stock_with_indicators.csv")

# Initialize the trading environment
env = StockTradingEnv(aapl_df)
obs, _ = env.reset()  # `gymnasium` requires reset() to return (obs, info)

# Run the trained model in the environment
for _ in range(len(aapl_df)):
    action, _states = model.predict(obs)
    obs, reward, terminated, truncated, _ = env.step(action)  # `gymnasium` returns 5 values
    env.render()
    
    if terminated or truncated:
        break



Step: 31, Balance: 10000.00, Portfolio Value: 10000.00
Step: 32, Balance: 10000.00, Portfolio Value: 10000.00
Step: 33, Balance: 10000.00, Portfolio Value: 10000.00
Step: 34, Balance: 10000.00, Portfolio Value: 10000.00
Step: 35, Balance: 10000.00, Portfolio Value: 10000.00
Step: 36, Balance: 10000.00, Portfolio Value: 10000.00
Step: 37, Balance: 10000.00, Portfolio Value: 10000.00
Step: 38, Balance: 10000.00, Portfolio Value: 10000.00
Step: 39, Balance: 10000.00, Portfolio Value: 10000.00
Step: 40, Balance: 10000.00, Portfolio Value: 10000.00
Step: 41, Balance: 10000.00, Portfolio Value: 10000.00
Step: 42, Balance: 10000.00, Portfolio Value: 10000.00
Step: 43, Balance: 10000.00, Portfolio Value: 10000.00
Step: 44, Balance: 10000.00, Portfolio Value: 10000.00
Step: 45, Balance: 10000.00, Portfolio Value: 10000.00
Step: 46, Balance: 10000.00, Portfolio Value: 10000.00
Step: 47, Balance: 10000.00, Portfolio Value: 10000.00
Step: 48, Balance: 10000.00, Portfolio Value: 10000.00
Step: 49, 

In [10]:
import numpy as np

# load model
model = SAC.load("sac_NVDA")

# Validating
obs, _ = env.reset()
done = False
total_reward = 0
while not done:
    action, _ = model.predict(obs, deterministic=True) 
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated
    total_reward += reward

print(f"Final Porfits: {total_reward}")

Final Porfits: 11.79904389518406
